## Cargamos Librerias

In [252]:
from sklearn.model_selection import train_test_split

#Weak Modeles
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

#Metrics
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import StandardScaler #Estandarizador

from sklearn.utils import resample #para bootstraping.


import pandas as pd
import numpy as np

## Cargamos los Datos

In [253]:
dataset = pd.read_csv("heart.csv")
dataset

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


## Estandarización de Variables Numericas

In [254]:
df_scaled = dataset.copy()
cols_to_scale = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak'] 

features = df_scaled[cols_to_scale]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

df_scaled[cols_to_scale] = features
df_scaled

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0.952197,1,3,0.763956,-0.256334,1,0,0.015443,0,1.087338,0,0,1,1
1,-1.915313,1,2,-0.092738,0.072199,0,1,1.633471,0,2.122573,0,0,2,1
2,-1.474158,0,1,-0.092738,-0.816773,0,0,0.977514,0,0.310912,2,0,2,1
3,0.180175,1,1,-0.663867,-0.198357,0,1,1.239897,0,-0.206705,2,0,2,1
4,0.290464,0,0,-0.663867,2.082050,0,1,0.583939,1,-0.379244,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.290464,0,0,0.478391,-0.101730,0,1,-1.165281,1,-0.724323,1,0,3,0
299,-1.033002,1,3,-1.234996,0.342756,0,1,-0.771706,0,0.138373,1,0,3,0
300,1.503641,1,0,0.706843,-1.029353,1,1,-0.378132,0,2.036303,1,2,3,0
301,0.290464,1,0,-0.092738,-2.227533,0,1,-1.515125,1,0.138373,1,1,3,0


## Seleccionamos Variable Target y Variables Predictoras

In [255]:
#X = dataset.loc[:, dataset.columns != 'target']
#y = dataset.loc[:, dataset.columns == 'target']

X = df_scaled.drop('target',axis = 1)
y = df_scaled.target

## Separamos en data para Entrenamiento y Prueba

In [256]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2020)

## Creamos Configuración de Modelos

In [257]:
lr = 'LogisticRegression().fit(X_train, y_train)'
svm = 'SVC(gamma="scale", kernel="rbf").fit(X_train, y_train)'
dt = 'DecisionTreeClassifier(criterion="gini", max_depth=4).fit(X_train, y_train)'
knn = 'KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)'
nb = 'GaussianNB().fit(X_train, y_train)'
lda = 'LinearDiscriminantAnalysis(solver="svd", store_covariance=True).fit(X_train, y_train)'
qda = 'QuadraticDiscriminantAnalysis(store_covariance=True).fit(X_train, y_train)'

models_list = {"lr" :lr, "svm":svm, "dt": dt,  "knn":knn, "nb":nb, "lda":lda, "qda":qda}
models_list

{'lr': 'LogisticRegression().fit(X_train, y_train)',
 'svm': 'SVC(gamma="scale", kernel="rbf").fit(X_train, y_train)',
 'dt': 'DecisionTreeClassifier(criterion="gini", max_depth=4).fit(X_train, y_train)',
 'knn': 'KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)',
 'nb': 'GaussianNB().fit(X_train, y_train)',
 'lda': 'LinearDiscriminantAnalysis(solver="svd", store_covariance=True).fit(X_train, y_train)',
 'qda': 'QuadraticDiscriminantAnalysis(store_covariance=True).fit(X_train, y_train)'}

## Creamos Dataset Nuevamente

In [258]:
dataset = pd.concat([X_train, y_train], axis=1)
dataset

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
204,0.841908,0,0,1.620649,-1.589792,0,0,-0.203210,0,4.451851,0,3,3,0
221,0.069886,1,0,0.478391,-0.565541,0,1,-1.690047,1,3.934233,0,0,3,0
66,-0.371269,1,2,-1.806125,-0.468914,0,1,-0.290671,1,0.138373,1,0,2,1
95,-0.150692,1,0,0.592617,-0.391612,0,0,-1.690047,1,-0.896862,2,0,3,1
170,0.180175,1,2,-0.092738,0.188152,1,0,-0.334401,1,-0.379244,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,-0.591847,0,1,0.135714,0.478035,0,1,0.540209,0,-0.896862,1,0,2,1
125,-2.246179,0,1,-0.778093,-0.700820,0,1,1.852124,0,-0.292975,2,0,2,1
91,0.290464,1,0,0.021488,-0.758796,0,1,0.802592,1,-0.896862,2,0,3,1
195,0.511041,1,0,2.191778,1.540936,0,0,-0.421862,1,2.036303,0,0,3,0


## Creamos Boost y Modelos

In [259]:
train_models_list = {}

for model_name, model in models_list.items():
    boot = resample(dataset, replace=True, n_samples=200, random_state=2020)
    X_train = boot.drop('target', axis = 1)
    y_train = boot.target
    train_model = eval(model)
    train_models_list[model_name] = train_model
    
train_models_list

{'lr': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 'svm': SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
     max_iter=-1, probability=False, random_state=None, shrinking=True,
     tol=0.001, verbose=False),
 'dt': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=4, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        

## Realizamos Predicciones con cada Modelo

In [260]:
auc_scores = {}
results_matrix = pd.DataFrame(columns=np.arange(0, len(y_test), 1).tolist())

for model_name, train_model in train_models_list.items():
    predicciones = train_model.predict(X_test)
    auc = roc_auc_score(y_test, predicciones)
    auc_scores[model_name] = auc
    tempDf = pd.DataFrame(predicciones).T
    results_matrix = results_matrix.append(tempDf)
    
    
    
results_matrix.index=list(train_models_list.keys())

 
results_matrix

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
lr,0,1,0,1,1,1,1,0,0,1,...,1,1,1,0,0,0,1,0,1,1
svm,0,1,0,1,1,1,1,0,0,1,...,1,1,1,0,0,0,1,1,1,1
dt,1,0,1,0,1,1,1,0,0,1,...,0,1,1,0,0,1,1,1,1,1
knn,0,1,0,1,1,1,1,0,0,1,...,1,1,1,0,0,0,1,1,1,1
nb,0,1,0,0,1,1,1,0,0,0,...,1,1,1,0,0,0,1,1,1,1
lda,0,1,0,0,1,1,1,0,0,1,...,1,1,1,0,0,0,1,0,1,1
qda,0,1,1,1,1,1,1,0,0,1,...,0,0,1,0,0,0,1,1,1,1


## Ralizamos Votación resultados

In [261]:
votacion = results_matrix.apply(pd.value_counts)
votacion

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,6,1,5,3,NaN,NaN,NaN,7.0,7.0,1,...,2,1,NaN,7.0,7.0,6,NaN,2,NaN,NaN
1,1,6,2,4,7.0,7.0,7.0,NaN,NaN,6,...,5,6,7.0,NaN,NaN,1,7.0,5,7.0,7.0


In [262]:
final_predictions = []

for (columnName, columnData) in votacion.iteritems():
    column_result = columnData.values
    final_predictions.append(np.nanargmax(column_result, axis=0))

final_predictions

[0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1]

In [263]:
auc = roc_auc_score(y_test, final_predictions)
auc

0.8175872093023255

In [264]:
auc_scores

{'lr': 0.8280038759689924,
 'svm': 0.8059593023255814,
 'dt': 0.7294089147286821,
 'knn': 0.8163759689922482,
 'nb': 0.840843023255814,
 'lda': 0.8175872093023255,
 'qda': 0.8120155038759691}